# State change propagation

In [1]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr
from bmcs_utils.model_notification import ModelNotifyMixin

Each model can be regarded as a tree of model components.
Let us define models `TopModel`, `InterimModel` and `SubModel`.

In [2]:
class SubModel(ModelNotifyMixin):
    name = 'submodel'
    length = bu.Float(2, GEO=True)

    @tr.observe('state_changed')
    def submodel_changed(self, event):
        print('submodel changed')

    ipw_view = bu.View(
        bu.Item('length')
    )

In [3]:
sm = SubModel(state_change_debug=True)

In [4]:
sm.length = 3

state_changed <__main__.SubModel object at 0x7f1546108810> TraitChangeEvent(object=<__main__.SubModel object at 0x7f1546108810>, name='length', old=2, new=3.0)
submodel changed


The change of a model parameter is observed and notified
by an event `sm.state_changed`. Let us observe this event
using a function


After executing the above cell, change the length in the
above model window to see the spy in action.

How did it happen? The `length` parameter of the `SubModel`
contains the metadate `GEO=True`. This indicates the type of
model input data. Except of `GEO` there are also the following
types of input data:

 - `BC` - boundary condition
 - `MAT` - material parameter
 - `DSC` - discretization
 - `CS` - cross section
 - `ALG` - algorithm parameter
 - `GEO` - global geometry

Change of any parameter denoted by one of the above tags
is observed and propagated along the model tree.

Let us define a more complex model

In [5]:
class InterimModel(ModelNotifyMixin):
    sm = bu.Instance(SubModel,(), GEO=True)
    stiffness = bu.Float(400, GEO=True)

    children = ['sm']
    @tr.observe('state_changed')
    def submodel_changed(self, event):
        print('interim model changed')


In [6]:
im = InterimModel(sm=sm, state_change_debug=True)

interim model changed


In [7]:
im.sm.length = 8

state_changed <__main__.SubModel object at 0x7f1546108810> TraitChangeEvent(object=<__main__.SubModel object at 0x7f1546108810>, name='length', old=3.0, new=8.0)
submodel changed
state_changed <__main__.InterimModel object at 0x7f15742d2180> Notification from child <__main__.SubModel object at 0x7f1546108810>
interim model changed


In [8]:
class TopLevelModel(ModelNotifyMixin):
    im = bu.Instance(InterimModel,(), DSC=True)
    time = bu.Float(400, ALG=True)

    children = ['im']
    @tr.observe('state_changed')
    def toplevel_changed(self, event):
        print('toplevel model changed')



In [9]:
tl = TopLevelModel(im=im, state_change_debug=True)
tl.im.state_change_debug = True
tl.im.sm.state_change_debug = True

toplevel model changed


In [10]:
tl.im.parents

{<__main__.TopLevelModel at 0x7f1545d43ea0>}

In [11]:
tl.im.sm.length = 10

state_changed <__main__.SubModel object at 0x7f1546108810> TraitChangeEvent(object=<__main__.SubModel object at 0x7f1546108810>, name='length', old=8.0, new=10.0)
submodel changed
state_changed <__main__.InterimModel object at 0x7f15742d2180> Notification from child <__main__.SubModel object at 0x7f1546108810>
interim model changed
state_changed <__main__.TopLevelModel object at 0x7f1545d43ea0> Notification from child <__main__.InterimModel object at 0x7f15742d2180>
toplevel model changed


In [12]:
tl.im = InterimModel(state_change_debug=True)

state_changed <__main__.TopLevelModel object at 0x7f1545d43ea0> TraitChangeEvent(object=<__main__.TopLevelModel object at 0x7f1545d43ea0>, name='im', old=<__main__.InterimModel object at 0x7f15742d2180>, new=<__main__.InterimModel object at 0x7f1545f8c9a0>)
toplevel model changed
parent <__main__.TopLevelModel object at 0x7f1545d43ea0> changing child from child <__main__.InterimModel object at 0x7f15742d2180> to <__main__.InterimModel object at 0x7f1545f8c9a0>
im


In [13]:
tl.im.sm.length = 11

submodel changed
state_changed <__main__.InterimModel object at 0x7f1545f8c9a0> Notification from child <__main__.SubModel object at 0x7f15742d4180>
interim model changed
state_changed <__main__.TopLevelModel object at 0x7f1545d43ea0> Notification from child <__main__.InterimModel object at 0x7f1545f8c9a0>
toplevel model changed


In [14]:
im.sm.length = 11

state_changed <__main__.SubModel object at 0x7f1546108810> TraitChangeEvent(object=<__main__.SubModel object at 0x7f1546108810>, name='length', old=10.0, new=11.0)
submodel changed
state_changed <__main__.InterimModel object at 0x7f15742d2180> Notification from child <__main__.SubModel object at 0x7f1546108810>
interim model changed


In [15]:
sm.length =12

state_changed <__main__.SubModel object at 0x7f1546108810> TraitChangeEvent(object=<__main__.SubModel object at 0x7f1546108810>, name='length', old=11.0, new=12.0)
submodel changed
state_changed <__main__.InterimModel object at 0x7f15742d2180> Notification from child <__main__.SubModel object at 0x7f1546108810>
interim model changed


# Test a list

In [16]:
class CrossSectionLayout(bu.ModelList):
    name = 'Cross Section Layout'

    items = bu.List(SubModel, [])

    def add_layer(self, rl):
        self.items.append(rl)

In [17]:
csl = CrossSectionLayout(state_change_debug=True)

In [18]:
csl.add_layer(SubModel(state_change_debug=True))

LIST STATE CHANGED TraitChangeEvent(object=<__main__.CrossSectionLayout object at 0x7f0620c86450>, name='items_items', old=<undefined>, new=TraitListEvent(index=0, removed=[], added=[<__main__.SubModel object at 0x7f0620d0dc20>]))


In [19]:
csl.items[0].name

'submodel'

In [20]:
csl.update_observers()

AttributeError: 'SubModel' object has no attribute 'get_tree_subnode'

In [21]:
csl.items[0].length = 3

state_changed <__main__.SubModel object at 0x7f0620d0dc20> TraitChangeEvent(object=<__main__.SubModel object at 0x7f0620d0dc20>, name='length', old=2, new=3.0)
submodel changed
